In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas import json_normalize
from requests.auth import HTTPBasicAuth

from dotenv.main import load_dotenv
import os

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

import seaborn as sns

from sklearn.preprocessing import LabelEncoder

import nltk
from nltk.tokenize import word_tokenize

import math

from sklearn.preprocessing import StandardScaler

import pickle


pd.options.display.max_seq_items = 20000
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 2000)

In [2]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from collections import Counter
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from imblearn.under_sampling import NearMiss


In [3]:
df_mode=pd.read_pickle("/Users/klaus/Documents/Jarvis/Second Try/Data/df_mode_30.11")

Gradient Boosting Trees for Funding

In [ ]:
df_mode.reset_index(drop=True)

In [86]:

df_fund=df_mode.loc[(df_mode["Y"]==0) | (df_mode["Y"]==4)].copy()
df_fund.loc[(df_mode["Y"]==0),"Y"]=0 #compare it to companies where nothing happened 
df_fund.loc[(df_mode["Y"]==4),"Y"]=1
df_fund.reset_index(drop=True)
X=df_fund.drop(columns=["name","Y"])
y=df_fund["Y"]


In [87]:
undersample = NearMiss(version=1, n_neighbors=3)
# transform the dataset
X, y = undersample.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [51]:
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'max_delta_step': [0,1,2,4],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000],
           'min_child_weight': [1,2,3,4]}
xgbr = xgb.XGBClassifier()
clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring='f1',
                         n_iter=20,
                         verbose=1)

In [54]:
clf.fit(X_train, y_train)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters: {'subsample': 0.8999999999999999, 'n_estimators': 100, 'min_child_weight': 4, 'max_depth': 5, 'max_delta_step': 0, 'learning_rate': 0.1, 'colsample_bytree': 0.8999999999999999, 'colsample_bylevel': 0.6}
Lowest RMSE:  nan


/var/folders/k0/ldv3tm456fl7jrgd0xn0vy5h0000gn/T/ipykernel_7377/3937026439.py:3: RuntimeWarning: invalid value encountered in double_scalars
  print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))


In [60]:
fix_params= clf.best_params_
fix_params["objective"] = 'binary:logistic'

In [88]:
xgb_fund = xgb.XGBClassifier(**fix_params)
xgb_fund = xgb_fund.fit(X_train, y_train)
y_pred = xgb_fund.predict(X_test)
y_proba= xgb_fund.predict_proba(X_test)

Gradient Boosting Trees for IPO, Acquired, Failed 

In [ ]:
df_mode.reset_index(drop=True)

In [97]:
df_fund=df_mode.loc[(df_mode["Y"]!=0) & (df_mode["Y"]!=4)].copy()
df_fund.loc[(df_mode["Y"]==1),"Y"]=0 #compare it to companies where nothing happened 
df_fund.loc[(df_mode["Y"]==2),"Y"]=1
df_fund.loc[(df_mode["Y"]==3),"Y"]=2
df_fund.reset_index(drop=True)
X=df_fund.drop(columns=["name","Y"])
y=df_fund["Y"]

In [98]:
sm = SMOTE(random_state = 2)
X,y = sm.fit_resample(X,y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [67]:
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'max_delta_step': [0,1,2,4],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000],
           'min_child_weight': [1,2,3,4]}
xgbr = xgb.XGBClassifier()
clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring=scorers["f1_score"],
                         n_iter=10,
                         verbose=1,
                        )

In [28]:
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

In [29]:
scorers = {
            'f1_score': make_scorer(f1_score, average='micro')
          }

In [71]:
clf.fit(X_train, y_train)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters: {'subsample': 0.8999999999999999, 'n_estimators': 1000, 'min_child_weight': 4, 'max_depth': 20, 'max_delta_step': 1, 'learning_rate': 0.1, 'colsample_bytree': 0.8999999999999999, 'colsample_bylevel': 0.7}
Lowest RMSE:  nan


/var/folders/k0/ldv3tm456fl7jrgd0xn0vy5h0000gn/T/ipykernel_7377/3937026439.py:3: RuntimeWarning: invalid value encountered in double_scalars
  print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))


In [72]:
fix_params_ipo= clf.best_params_
fix_params_ipo["objective"] = 'multi:softproba'
fix_params_ipo["num_class"]=3

In [99]:
xgb_clf = xgb.XGBClassifier(**fix_params)
xgb_clf = xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)
y_proba= xgb_clf.predict_proba(X_test)

In [106]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89      2270
           1       0.89      0.89      0.89      2246
           2       0.94      0.91      0.92      2274

    accuracy                           0.90      6790
   macro avg       0.90      0.90      0.90      6790
weighted avg       0.90      0.90      0.90      6790

